In [15]:
!pip install pandas faker requests python-dotenv



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Desktop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
import pandas as pd
from faker import Faker
import random
import uuid
import requests
import json
import os
from dotenv import load_dotenv

In [23]:
fake = Faker('pt_BR')
leads = []

# Crie uma lista de cargos variados para a lógica de qualificação no n8n
cargos = [
    'Diretor de Vendas', 'Gerente de Marketing', 'Líder de Equipe',
    'Analista Financeiro', 'Especialista em TI', 'Assistente Administrativo',
    'Coordenador de Projetos', 'Cientista de Dados', 'Engenheiro de Software']

# Gera 100 leads
for i in range(3):
    # Campos obrigatórios
    lead_id = str(uuid.uuid4())
    nome = fake.name()
    email = fake.email()
    
    # Campos com chance de serem nulos
    telefone = fake.phone_number() if random.random() < 0.8 else None
    empresa = fake.company() if random.random() < 0.9 else None
    cargo = random.choice(cargos)
    nascimento = fake.date_of_birth(minimum_age=20, maximum_age=60).strftime('%Y-%m-%d')
    data_admissao = fake.date_this_decade().strftime('%Y-%m-%d')

    lead = {
        'id': lead_id,
        'nome': nome,
        'email': email,
        'telefone': telefone,
        'cargo': cargo,
        'empresa': empresa,
        'nascimento': nascimento,
        'data_admissao': data_admissao}
    leads.append(lead)

df = pd.DataFrame(leads)

In [24]:
df

,id,nome,email,telefone,cargo,empresa,nascimento,data_admissao
0,e96e0181-cdc6-43a6-80e9-be4460427689,Larissa Melo,fonsecalevi@example.org,None,Analista Financeiro,Lopes Ltda.,1974-03-26,2020-06-19
1,328f8547-31df-4d70-93ac-59c8adf89536,Rafael Lopes,matteoduarte@example.org,+55 84 3097 1580,Engenheiro de Software,Abreu e Filhos,1967-08-03,2020-08-22
2,ea343e1d-4849-45a4-9b21-9c393f4334ca,João Lucas Alves,costajoaquim@example.net,+55 (011) 7097 2521,Especialista em TI,Nascimento - ME,1990-08-24,2022-07-26


In [25]:
load_dotenv()

webhook_url = os.getenv('N8N_WEBHOOK_URL')

if not webhook_url:
    print("Erro: A variável de ambiente N8N_WEBHOOK_URL não foi encontrada. Verifique o seu arquivo .env.")
else:
    # Itera sobre cada linha (lead) do DataFrame e envia para o Webhook
    for index, row in df.iterrows():
        lead_json = row.to_dict()
        
        try:
            response = requests.post(webhook_url, data=json.dumps(lead_json), headers={'Content-Type': 'application/json'})
            
            if response.status_code == 200:
                print(f"Lead {index+1} enviado com sucesso. Resposta: {response.text}")
            else:
                print(f"Erro ao enviar lead {index+1}. Status: {response.status_code}, Resposta: {response.text}")

        except requests.exceptions.RequestException as e:
            print(f"Erro de conexão ao enviar lead {index+1}: {e}")

Lead 1 enviado com sucesso. Resposta: {"message":"Workflow was started"}
Lead 2 enviado com sucesso. Resposta: {"message":"Workflow was started"}
Lead 3 enviado com sucesso. Resposta: {"message":"Workflow was started"}
